## Upload Reference Data to Redis 


This notebook uploads reference data to our Memorystore reference data Redis instance. 

In [1]:
from google.cloud import bigquery
import os
import pandas as pd
import pickle
import redis
import tqdm
#import dbtypes

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jupyter/creds.json"
bq_client = bigquery.Client()
project = "eng-reactor-287421"

In [3]:
def sqltodf(sql,limit = ""):
    if limit != "": 
        limit = f" ORDER BY RAND() LIMIT {limit}"
    bqr = bq_client.query(sql + limit).result()
    return bqr.to_dataframe()

The following query uses joins to retrive for any cusip only the record with the latest trade and the largest RTRS control number. Filtering on largest RTRS control number is necessary because trades sometimes occur simultaneously, and we want to select which of the simultaneous trades is selected in a predictable way. Later we may decide to select only those trades which are economically significant. The Cloud Function that updates our Redis database uses a simpler query with windowing to achieve a similar result becuase of the computing limitations BigQuery when it comes to self-joins in views.

In [4]:
%%time
df = sqltodf("select * from auxiliary_views.trade_history_latest_ref_data_minimal_exclusions where date(recent[safe_offset(0)].publish_datetime) = current_date('America/New_York')")

CPU times: user 4.36 s, sys: 1.78 s, total: 6.14 s
Wall time: 1min 36s


In [ ]:
df.head()

In [5]:
redis_client = redis.Redis(host='10.14.140.37', port=6379, db=0)

In [6]:
def upload_data_to_redis(key, value):
    value = pickle.dumps(value,protocol=pickle.HIGHEST_PROTOCOL)
    redis_client.set(key, value)

In [7]:
redis_client.dbsize()

1356391

In [8]:
redis_client.exists("64971XQM3")

1

In [9]:
for _,row in tqdm.tqdm(df.iterrows(), total=len(df)):
    key = row['cusip']
    try:
        upload_data_to_redis(key,row)
    except Exception as e:
        raise e

100%|██████████| 18043/18043 [12:07<00:00, 24.80it/s]


In [9]:
#Retriving data for a single cusip
x = redis_client.mget(['732203BJ2','732203BJ2'])

In [32]:
upload_data_to_redis("hi", "hi back")

In [32]:
x = pickle.loads(redis_client.get('74442PB38'))

In [ ]:
x

In [49]:
import numpy as np

In [57]:
x["recent"][0]["yield"] = None

In [58]:
x["recent"][0]["yield"]

In [59]:
x["cusip"] = "QQQQQQQQQ"

In [63]:
x["recent"]

array([{'msrb_valid_from_date': datetime.datetime(2023, 7, 3, 10, 14, 46), 'msrb_valid_to_date': datetime.datetime(2100, 1, 1, 0, 0), 'rtrs_control_number': 2023070301511900, 'trade_datetime': datetime.datetime(2023, 7, 3, 10, 14, 10), 'publish_datetime': datetime.datetime(2023, 7, 3, 10, 14, 46), 'yield': nan, 'dollar_price': 98.562, 'par_traded': Decimal('1730000.000000000'), 'trade_type': 'D', 'is_non_transaction_based_compensation': False, 'is_lop_or_takedown': False, 'brokers_broker': None, 'is_alternative_trading_system': True, 'is_weighted_average_price': False, 'settlement_date': datetime.date(2023, 7, 6), 'calc_date': datetime.date(2045, 5, 1), 'calc_day_cat': 2, 'maturity_date': datetime.date(2045, 5, 1), 'next_call_date': datetime.date(2030, 11, 1), 'par_call_date': datetime.date(2030, 11, 1), 'refund_date': None},
       {'msrb_valid_from_date': datetime.datetime(2023, 7, 3, 10, 15, 11), 'msrb_valid_to_date': datetime.datetime(2100, 1, 1, 0, 0), 'rtrs_control_number': 20230

In [54]:
key = x['cusip']
upload_data_to_redis(key,x)

In [1]:
with open('6-30-neg-yield.csv') as f:
    new = pd.read_csv(f)

In [ ]:
new

,cusip
0,468312EU3
1,16876QBG3
2,35981VAK6
3,235308QR7
4,706877AD9
...,...
1041,455719CP7
1042,76856PBF5
1043,424682DV1
1044,262588HC3


In [ ]:
redis_client.get("64972GBW3")

In [39]:
redis_client.delete("741530HY8")

1

In [15]:
for cusip in new["cusip"]:
    x = redis_client.delete(cusip)

In [19]:
x = pickle.loads(redis_client.get('64972GBW3'))

TypeError: a bytes-like object is required, not 'NoneType'

In [64]:
redis_client.exists("488386JA2")

1